# Immigration data

In [140]:
import pandas as pd
from IPython.display import Markdown;
import numpy

## immigration_citizenship.tsv

In [141]:
url = "../../Data/immigration/immigration_citizenship.tsv"
table_raw = pd.read_csv(url,sep='\t')
year_range = range(1998,2024,1)


In [142]:

age = []
agedef = []
birth_country = []
sex = []
country = []

for index,row in table_raw.iterrows():
    title_split = (row[0].split(","))
    

    age_str = title_split[1][1:]
    
    if age_str.isnumeric():
        age.append(age_str)

    else:
        if title_split[1]=="Y_GE100":
            age.append(str(100))
        elif title_split[1]=="Y_LT1":
            age.append(str(0))
        else:
            age.append(title_split[1]) 

    agedef.append(title_split[2])
    birth_country.append(title_split[3])
    sex.append(title_split[5])
    country.append(title_split[6])
    
    

table_raw["age"]= age
table_raw["agedef"] = agedef
table_raw["birth_country"] = birth_country
table_raw["sex"]=sex
table_raw["country"]=country

table_raw.drop(columns=["freq,age,agedef,citizen,unit,sex,geo\TIME_PERIOD"],inplace=True)
display(table_raw.head())

<>:37: SyntaxWarning: invalid escape sequence '\T'
<>:37: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipykernel_21217/4017857481.py:37: SyntaxWarning: invalid escape sequence '\T'
  table_raw.drop(columns=["freq,age,agedef,citizen,unit,sex,geo\TIME_PERIOD"],inplace=True)
/tmp/ipykernel_21217/4017857481.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title_split = (row[0].split(","))


,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,...,2019,2020,2021,2022,2023,age,agedef,birth_country,sex,country
0,28638,35797,:,34825,39696 b,44245,48664,45613,38905,31751,...,46329,42715,46214,97378,64503,TOTAL,COMPLET,FOR_STLS,F,AT
1,29382,33021,31296,39605,40608,40118,42604,48787,49847,52652,...,59833,47172,55678,93814,71027,TOTAL,COMPLET,FOR_STLS,F,BE
2,:,:,:,:,:,:,:,:,:,25,...,6588 p,6190 p,6852,8357,18085,TOTAL,COMPLET,FOR_STLS,F,BG
3,36508,:,42245,:,:,:,49223,47344,50424,65014,...,57407,53380,57705,66610,117356,TOTAL,COMPLET,FOR_STLS,F,CH
4,:,:,:,:,:,:,:,:,:,:,...,11953,12580,11222,15632,19850 b,TOTAL,COMPLET,FOR_STLS,F,CY


#### Reindexing, renaming columns, adding ISO3 codes


In [143]:
#rename country codes
country_codes = pd.read_csv("../../Data/country-region-codes.csv")
new_columns = {}

#numeric columns
for column in table_raw.columns:
    if column.strip().isnumeric():
        new_columns[column] = int(column.strip())
    else:
        new_columns[column]=column
        
table = table_raw.rename(columns=new_columns)
table.rename(columns={"country":"alpha-2"},inplace=True)

table = pd.merge(table,country_codes,on="alpha-2").drop(columns=["country-code","alpha-2"]).rename(columns={"name":"Country","alpha-3":"ISO3"})

#reordering

new_order = list(table.columns[-6:])+[i for i in year_range]# IMPORTANT to set -6 to number of non numeric columns

table = table[new_order]
display(table.head())

,age,agedef,birth_country,sex,Country,ISO3,1998,1999,2000,2001,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,TOTAL,COMPLET,FOR_STLS,F,Austria,AUT,28638,35797,:,34825,...,48788,65441,55097,48739,45444,46329,42715,46214,97378,64503
1,TOTAL,COMPLET,FOR_STLS,F,Belgium,BEL,29382,33021,31296,39605,...,51236,56379,50222,51820,55655,59833,47172,55678,93814,71027
2,TOTAL,COMPLET,FOR_STLS,F,Bulgaria,BGR,:,:,:,:,...,7295 p,6276,5522,5844 p,5960 p,6588 p,6190 p,6852,8357,18085
3,TOTAL,COMPLET,FOR_STLS,F,Switzerland,CHE,36508,:,42245,:,...,62616,61336,59968,57238,57673,57407,53380,57705,66610,117356
4,TOTAL,COMPLET,FOR_STLS,F,Cyprus,CYP,:,:,:,:,...,5630,7362,7437,9498,10230,11953,12580,11222,15632,19850 b


#### Fixing values
Renaming : to NaN
fixing provisional data

In [144]:
replaced = table.copy()

#fix numeric values
for column in replaced.columns:
    replaced[column] = replaced[column].apply(str.strip)
    # print(type(replaced[column][0]))

replaced.replace(to_replace=":",value=numpy.nan,inplace=True)

#tags
tags = ["b","p","ep","e","be","bep","P"]

#I used Deepseek AI for this, it was quite complex
pattern = r'\b(?:' + '|'.join(tags) + r')\b'

# Apply the replacement to all columns (or specify specific columns)
df = replaced.apply(lambda x: x.str.replace(pattern, '', regex=True).str.strip())



#### Saving to file

In [ ]:
path = "../../Data/immigration/immigration_citizenship.csv"

display(df.head())
# df.to_csv(path,index=False)

,age,agedef,birth_country,sex,Country,ISO3,1998,1999,2000,2001,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,TOTAL,COMPLET,FOR_STLS,F,Austria,AUT,28638,35797,NaN,34825,...,48788,65441,55097,48739,45444,46329,42715,46214,97378,64503
1,TOTAL,COMPLET,FOR_STLS,F,Belgium,BEL,29382,33021,31296,39605,...,51236,56379,50222,51820,55655,59833,47172,55678,93814,71027
2,TOTAL,COMPLET,FOR_STLS,F,Bulgaria,BGR,NaN,NaN,NaN,NaN,...,7295,6276,5522,5844,5960,6588,6190,6852,8357,18085
3,TOTAL,COMPLET,FOR_STLS,F,Switzerland,CHE,36508,NaN,42245,NaN,...,62616,61336,59968,57238,57673,57407,53380,57705,66610,117356
4,TOTAL,COMPLET,FOR_STLS,F,Cyprus,CYP,NaN,NaN,NaN,NaN,...,5630,7362,7437,9498,10230,11953,12580,11222,15632,19850


## immigration_country_born.tsv

In [146]:
url = "../../Data/immigration/immigration_country_born.tsv"
table_raw = pd.read_csv(url,sep='\t')
year_range = range(2008,2024,1)

age = []
agedef = []
birth_country = []
sex = []
country = []

for index,row in table_raw.iterrows():
    title_split = (row[0].split(","))
    
    age_str = title_split[1][1:]
    
    if age_str.isnumeric():
        age.append(age_str)

    else:
        if title_split[1]=="Y_GE100":
            age.append(str(100))
        elif title_split[1]=="Y_LT1":
            age.append(str(0))
        else:
            age.append(title_split[1]) 

    agedef.append(title_split[2])
    birth_country.append(title_split[3])
    sex.append(title_split[5])
    country.append(title_split[6])
    
    

table_raw["age"]= age
table_raw["agedef"] = agedef
table_raw["birth_country"] = birth_country
table_raw["sex"]=sex
table_raw["country"]=country

table_raw.drop(columns=["freq,age,agedef,c_birth,unit,sex,geo\TIME_PERIOD"],inplace=True)

<>:40: SyntaxWarning: invalid escape sequence '\T'
<>:40: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipykernel_21217/3853864483.py:40: SyntaxWarning: invalid escape sequence '\T'
  table_raw.drop(columns=["freq,age,agedef,c_birth,unit,sex,geo\TIME_PERIOD"],inplace=True)
/tmp/ipykernel_21217/3853864483.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title_split = (row[0].split(","))


#### Reindexing, renaming columns, adding ISO3 codes


In [147]:
#rename country codes
country_codes = pd.read_csv("../../Data/country-region-codes.csv")
new_columns = {}

#numeric columns
for column in table_raw.columns:
    if column.strip().isnumeric():
        new_columns[column] = int(column.strip())
    else:
        new_columns[column]=column
        
table = table_raw.rename(columns=new_columns)
table.rename(columns={"country":"alpha-2"},inplace=True)

table = pd.merge(table,country_codes,on="alpha-2").drop(columns=["country-code","alpha-2"]).rename(columns={"name":"Country","alpha-3":"ISO3"})

#reordering
new_order = list(table.columns[-6:])+[i for i in year_range]

table = table[new_order]
# display(table.head())

#### Fixing values
Renaming : to NaN
fixing provisional data

In [148]:
replaced = table.copy()

#fix numeric values
for column in replaced.columns[3:]:
    replaced[column] = replaced[column].apply(str.strip)
    # print(type(replaced[column][0]))

replaced.replace(to_replace=":",value=numpy.nan,inplace=True)

#tags
tags = ["b","p","ep","e","be","bep","P"]

#I used Deepseek AI for this, it was quite complex
pattern = r'\b(?:' + '|'.join(tags) + r')\b'

# Apply the replacement to all columns (or specify specific columns)
df = replaced.apply(lambda x: x.str.replace(pattern, '', regex=True).str.strip())



#### Saving to file

In [ ]:
path = "../../Data/immigration/immigration_country_born.csv"

# df.to_csv(path,index=False)